# Fine-tuning mT5 pour la summarisation en français

Ce notebook présente étape par étape le fine-tuning d'un modèle mT5 pour générer des résumés en français à partir du jeu de données MLSUM.

## Installation des dépendances

In [ ]:
!pip install transformers datasets rouge_score torch sentencepiece evaluate
!python -m spacy download fr_core_news_sm

## Import des bibliothèques

In [ ]:
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    TrainingArguments,
    Trainer
)
import evaluate

## Chargement et préparation du jeu de données

In [ ]:
# Chargement du jeu de données MLSUM en français
# A peu près 300.000 exemples
dataset = load_dataset("mlsum", "fr")

# Création d'un subset de 10 000 exemples avec split train/test
# A augmenter pour un modèle bien entrainé
subset = dataset["train"].select(range(10000)).train_test_split(
    test_size=0.2,
    seed=42
)

#8000 pour entrainer
print(subset["train"])

#2000 pour tester // ces exemples va plûtot m'ont servir pour évaluation
print(subset["test"])

## Configuration des paramètres

In [ ]:
MODEL = 'google/mt5-base'
BATCH_SIZE = 2 # A modifier si entrainement sur cpu
NUM_PROCS = 4
EPOCHS = 10 # A modifier si entrainement sur cpu
MAX_LENGTH = 768
OUT_DIR = 'résultat'

## Tokenizer et fonction de prétraitement

In [ ]:
# Initialisation du tokenizer pour le français
tokenizer = MT5Tokenizer.from_pretrained(MODEL)
tokenizer.src_lang = "fr_FR"
tokenizer.tgt_lang = "fr_FR"
tokenizer.add_tokens(["<2fr>"], special_tokens=True)

def preprocess_function(examples):
    inputs = [doc.replace("\n", " ").strip()[:2000] for doc in examples["text"]]
    targets = [doc.replace("\n", " ").strip()[:300] for doc in examples["summary"]]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding="max_length",
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

## Prétraitement des données

In [ ]:
# Tokenisation en parallèle
tokenized_datasets = subset.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS,
    remove_columns=subset["train"].column_names # garder que les champs tokenized
)

## Chargement du modèle

In [ ]:
# Chargement du modèle mT5
model = MT5ForConditionalGeneration.from_pretrained(MODEL)
model.to('cuda') #entrainement sur gpu

## Définition des métriques

In [ ]:
# Configuration de ROUGE
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Nettoyage pour le français
    decoded_preds = [p.replace("▁", " ").replace("  ", " ").strip() for p in decoded_preds]
    decoded_labels = [l.replace("▁", " ").replace("  ", " ").strip() for l in decoded_labels]

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    return {k: round(v, 4) for k, v in result.items()}

## Configuration de l'entraînement

In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    learning_rate=3e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir=f"{OUT_DIR}/logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    fp16=True,
    report_to="tensorboard"
)

## Création du Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Lancement de l'entraînement

In [ ]:
trainer.train()

## Génération de résumés et tests

In [ ]:
import random

def generate_summary(text):
    text = text.encode("utf-8", "ignore").decode("utf-8")[:2000]
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    print("\nTexte tokenisé:", tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True))
    forced_bos_token_id = tokenizer.convert_tokens_to_ids("<2fr>")
    summary_ids = model.generate(
        inputs.input_ids,
        max_length=150,
        num_beams=6,
        forced_bos_token_id=forced_bos_token_id,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0].cpu(), skip_special_tokens=True)

def test_model_samples(num_samples=3):
    test_samples = random.sample(range(len(subset["test"])), num_samples)
    for i in test_samples:
        text = subset["test"][i]["text"]
        reference = subset["test"][i]["summary"]
        generated_summary = generate_summary(text)
        print(f"\n=== Exemple {i+1}/{num_samples} ===")
        print(f"Texte original ({len(text)} caractères):\n{text[:500]}...")
        print(f"\nRésumé généré:\n{generated_summary}")
        print(f"\nRésumé référence:\n{reference}")
        print("="*50 + "\n")

# Exécution du test
test_model_samples(num_samples=3)